In [2]:
from pianoq.misc.mplc_lab_imports import *  

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

# Camera things

In [44]:
cam = PCOCamera()
cam.set_exposure_time(0.5)

# ml = MPLCAligner(use_cam=False)
# ml.update('1to5w4f', '5to11w8', pi_steps_x=[0], pi_steps_plane=1)

In [50]:
cam.set_exposure_time(2)

In [47]:
roi=(750, 350, 1200, 850)
live_cam(cam, roi=roi)

In [48]:
mplc.load_masks(masks)
roi=(750, 350, 1200, 850)
cam.show_image(roi=roi)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [60]:
roi=(700, 300, 1250, 900)
name = f'tilts_{tilts}.fits'
cam.save_image(path=rf'{dir_path}\{name}', roi=roi)
cam.show_image(roi=roi)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [ ]:
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = rf'{dir_path}\{timestamp}_{name}.cam'
cam.save_image(path)

In [61]:
cam.close()

# MPLC 

In [3]:
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

tilts = None 
mplc = MPLCDevice(linear_tilts=True, plane_10_tilts=tilts)
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_29_14_27_10_U1U.masks"  # with dead zone 
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\U1U_2024_09_10_14_00_34.masks"  # old masks  
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_22_15_50_U1U_minus_dist_plane10.masks"  # minus  

masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep)
# phases = np.random.uniform(0, 2*np.pi, size=50)
# masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks)
# mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=None)
mplc.restore_location()

Got MPLC


# hardware 

In [4]:
backlash = 0.0
wait_after_move = 0.3
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)
tt = get_timetagger(integration_time=1)

Got Zaber motors
Got Thorlabs motors


In [64]:
mxi.move_absolute(9.405)
myi.move_absolute(4.795)
print(mxi.get_position())
print(myi.get_position())

9.405
4.795


In [65]:
mxs.move_absolute(12.91)
mys.move_absolute(5.72)
print(mxs.get_position())
print(mys.get_position())

12.909994500000002
5.720000625


In [147]:
myi.move_relative(0.01)

In [4]:
print(mxi.get_position()) # 9.2701
print(myi.get_position()) # 3.3100

9.3
4.05


BS
- best idler: ~9K
- best signal: ~8.3K 

Mirror:
- best idler: 
- best signal: 


# New WFM 

## with defocus distance and dead zone 

In [99]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\{timestamp}_U1U_minus_dist_plane10.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_U1U_minus_dist_plane10.mplcsim'
mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=0, last_plane_extra_dist=-8.4e-3)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087 , 0.087 , 0.087 , 0.087 , 0.138 , 0.087 , 0.087 , 0.087 ,
       0.087 , 0.0786]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}


100%|██████████| 15/15 [03:52<00:00, 15.52s/it]


In [7]:
mplc_sim.res.show_overlap()

In [8]:
mplc_sim.res._calc_fidelity()
mplc_sim.res.fidelity

0.9896838134848069

In [9]:
mplc_sim.res.show_all()

# mark spot locations 

In [11]:
from pianoq_results.scan_result import ScanResult
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
locs_signal_path = fr'{dir_path}\{timestamp}_s2_sig_mirror_3.5.locs'
locs_idler_path = fr'{dir_path}\{timestamp}_s1_idl_mirror_3.5.locs'

res_idl = ScanResult(fr'{dir_path}\2024_10_06_12_06_17_idler_3_5_spots.scan')
res_sig = ScanResult(fr'{dir_path}\2024_10_06_12_07_31_signal_3_5_spots.scan')
# res_idl.show_singles()
# res_sig.show_singles()
res_idl.get_xys(1, saveto_path=locs_idler_path)
res_sig.get_xys(2, saveto_path=locs_signal_path)

Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_06_12_25_13_s1_idl_mirror_3.5.locs
Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_06_12_25_13_s2_sig_mirror_3.5.locs


[(12.984470529470531, 7.019139194139194),
 (12.948240093240095, 6.8358558108558105),
 (12.937584082584083, 6.462895437895438),
 (12.897091242091243, 6.077147852147852),
 (12.869385614385616, 5.697793872793873)]

# Phases

In [61]:
phases_result = PhaseFinderResult(fr"{dir_path}\2024_10_07_12_01_25_fine_new_mirror.phases")
phases_result.reload()
phases_result.integration_time

array(30)

In [95]:
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_14_17_06_fine_new_mirror_more.phases"
phases_path = None 
masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.load_masks(masks)

saveto_path = fr'{dir_path}\{tnow()}_fine_new_mirror_from_zero_more.phases'
integration_time = 50

phases_result = PhaseFinderResult(fr"{dir_path}\2024_10_07_16_32_20_fine_new_mirror_from_zero_long.phases")
initial_phases = phases_result.phases
# initial_phases = None 

pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=saveto_path,
                 modes_to_keep=modes_to_keep, intial_phases=initial_phases, coin_window=tt.coin_window, no_hw_mode=True)

pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

# locs 
locs_idl = np.load(fr'{dir_path}\2024_10_07_10_45_00_fine_s1_idl_mirror_3_5_spots.locs')['locs']
locs_sig = np.load(fr'{dir_path}\2024_10_07_10_45_00_fine_s2_sig_mirror_3_5_spots.locs')['locs']

# i,j of corr matrix that is supposed to be strong 
i = 1
j = 1
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])        
mplc.restore_location()
time.sleep(1)
pf.find_phases()

0,0: 18057.1, 11329.5, 4.4
0,1: 18058.4, 12049.4, 4.0
0,2: 18058.9, 12177.5, 3.0
0,3: 18020.0, 12265.7, 1.9
0,4: 17994.7, 12416.3, 2.2
0,5: 17940.5, 12366.8, 2.6
0,6: 17891.3, 12273.0, 4.3
0,7: 17978.5, 12208.1, 5.6
0,8: 17924.2, 12172.9, 6.0
0,9: 17943.0, 12147.2, 5.6
1,0: 17880.5, 12121.9, 5.1
1,1: 17860.7, 12181.1, 6.6
1,2: 17847.3, 12227.6, 5.4
1,3: 17881.9, 12317.8, 4.3
1,4: 17835.6, 12151.9, 2.5
1,5: 17827.2, 10845.7, 2.1
1,6: 17854.9, 12395.4, 1.6
1,7: 17741.7, 12248.0, 1.6
1,8: 17771.8, 12150.2, 2.9
1,9: 17782.1, 12115.9, 4.8
2,0: 17771.3, 12153.4, 6.5
2,1: 17695.8, 12393.9, 7.5
2,2: 17856.5, 12640.9, 8.3
2,3: 17832.0, 12721.2, 8.1
2,4: 17718.4, 12818.9, 8.2
2,5: 17825.2, 12953.9, 5.3
2,6: 17814.9, 12845.0, 4.4
2,7: 17768.3, 12684.3, 3.7
2,8: 17879.8, 12631.4, 4.7
2,9: 17818.6, 12451.1, 5.3
3,0: 17829.4, 12513.3, 8.8
3,1: 17746.5, 12428.3, 8.4
3,2: 17708.3, 12376.3, 8.8
3,3: 17758.0, 12459.2, 8.7
3,4: 17689.9, 12447.0, 8.3
3,5: 17682.1, 12428.8, 7.6
3,6: 17694.2, 12481.9, 8.4
3

# Correlations

In [98]:
backlash = 0.0
wait_after_move = 0.3
integration_time = 50
coin_window = 2e-9

masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_19_14_12_fine_new_mirror_from_zero_more.phases"
masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.load_masks(masks)

scan_path = fr'{dir_path}\{tnow()}.dscan'

locs_idl = np.load(fr'{dir_path}\2024_10_07_10_45_00_fine_s1_idl_mirror_3_5_spots.locs')['locs']
locs_sig = np.load(fr'{dir_path}\2024_10_07_10_45_00_fine_s2_sig_mirror_3_5_spots.locs')['locs']

dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window, no_hw_mode=True)

dps.m_sig_x, dps.m_sig_y, dps.m_idl_x, dps.m_idl_y, dps.time_tagger = mxs, mys, mxi, myi, tt
dps.time_tagger.set_integration_time(integration_time)
time.sleep(1)
dps.scan()

beginning scan
0, 0: 17290.34, 13691.80, 6.56
0, 1: 17276.22, 10941.08, 0.82
0, 2: 17414.30, 12765.58, 1.32
0, 3: 17341.82, 10046.16, 0.66
0, 4: 17360.40, 1516.72, 0.18
1, 0: 17858.38, 13703.56, 1.18
1, 1: 17849.86, 10947.82, 8.64
1, 2: 17933.68, 12816.48, 1.16
1, 3: 17908.62, 10081.94, 1.04
1, 4: 17908.88, 1528.02, 0.16
2, 0: 15755.76, 13701.48, 1.28
2, 1: 15735.66, 10975.88, 0.80
2, 2: 15803.86, 12824.80, 6.04
2, 3: 15828.36, 10121.18, 0.94
2, 4: 15751.80, 1534.06, 0.12
3, 0: 10459.34, 13731.34, 0.90
3, 1: 10526.68, 11045.04, 1.10
3, 2: 10523.24, 12846.90, 0.66
3, 3: 10474.96, 10119.80, 3.64
3, 4: 10532.82, 1525.44, 0.14
4, 0: 3033.84, 13833.74, 0.22
4, 1: 3040.68, 11087.70, 0.30
4, 2: 3046.56, 12889.06, 0.16
4, 3: 3049.80, 10202.60, 0.12
4, 4: 3043.92, 1532.70, 0.06


In [90]:
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_16_03_59_fine_new_mirror_from_zero.phases"
masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.load_masks(masks)

In [89]:
i = 1
j = 1
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])        

In [77]:
mxi.move_absolute(9.35)
myi.move_absolute(4.424)
print(mxi.get_position())
print(myi.get_position())

9.35
4.424


In [80]:
mxs.move_absolute(12.95)
mys.move_absolute(6.075)
print(mxs.get_position())
print(mys.get_position())

12.9499995
6.074997375000001


In [97]:
# dps.res.show_singles()
dps.res.show()
dps.res.real_coins.sum()

25.441713591

In [52]:
mplc.close()